# Sentiment Analysis for Bengali News Articles

_**Abstract**:_ With an exponential increase in digital information, we need automated methods to enable easier analysis for various applications, both commercial and otherwise. Sentiment Analysis is an important component of the analysis, [reasons], but the bulk of the work done is in English. Research in Bengali is severely hampered, primarily by the lack of publicly available quality datasets and tools. In this project I create a basic rule-based classifier for Bengali news reports. I perform **polarity classification** on the articles. The primary outputs of this project are:
    
1. A basic rule-based sentiment classifier, for polarity classification with decent precision
2. [Secondary] A rudimentary SentiWordNet for Bengali, using a method proposed by Das and Bandyopadhyay (2009)
3. [Secondary] A basic Bengali-English word-level dictionary
    

In [1]:
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords

import pandas as pd

import json
import csv
import pickle

import random as rd

In [2]:
# Global variables used by all
tdict = {}
dframe = pd.DataFrame(index=['body', 'label', 'link', 'title'])
eng_stopset = list(stopwords.words('english'))
eng_negs = ['not', 'n\'t', 'none', 'never', 'no', 'non']

## Misc functions

Functions I've used

### Tokenizer

A basic tokenizer using space-separation, as importing a library when all I would use was the tokenizer seemed rather unnecessary. In case more preprocessing is done with regard to stemming, etc, then we might import cltk or nltk in order to run the entire preprocessing suite.

In [3]:
def tokenize(string):
    rlist = []
    punctuation = [' ', '।', '\'', '\"', ',', '‘', '’', '—']
    word = ''
    for character in string:
        if character in punctuation:
            rlist.append(word)
            word = ''
        else:
            word += character
    
    return rlist

### Initializer

In [5]:
def initialize():
    global tdict
    global dframe
    
    with open('Dictionaries/bn_en_dict.pickle', 'rb') as handle:
        tdict = pickle.load(handle)
        
    dframe = pd.DataFrame(index=['body', 'label', 'link', 'title'])
    
initialize()

### Display

Used to verify precision.

In [31]:
def checkSenti():
    '''
    Uncomment to test
    '''
    
    global weightframe
    t = weightframe.transpose()
    return t[t.sentiment == 'positive'].sample(n = 10)
    
checkSenti()

,body,label,link,title,score,sentiment
art6656,উত্তর কলকাতার সরু গলি দিয়ে দৌড়ে ‘মত্ত’ যুবকক...,state,https://ebela.in/state/babul-supriyo-caught-a-...,দেড়শো মিটার দে ছুট! মত্ত যুবককে পাকড়ালেন মন্...,"[0.2398273601398601, 0.11828015734265736, 4.28...",positive
art3467,একসময় উর্দির সম্মান রক্ষাই তাঁর ব্রত ছিল। আজ ...,state,https://ebela.in/state/retired-police-officer-...,"পুলিশের মার কী, অবসরের পর বুঝলেন পুলিশকর্মী","[0.14612638815441564, 0.09014806980433633, 2.2...",positive
art5211,মাঝরাতে বাড়িতে আওয়াজ শুনে চমকে উঠেছিলেন এক ব...,national,https://ebela.in/national/noida-girl-arrested-...,মেয়ের ঘরে কে! দেখতে গিয়ে প্রাণ হারালেন প্রৌঢ়,"[0.1464078374455733, 0.09637961619093696, 3.17...",positive
art8026,নিহত আবেশ দাশগুপ্তের শরীর জুড়ে দেখতে পাওয়া গ...,state,https://ebela.in/state/28-injury-sign-in-abesh...,আবেশের দেহে ২৮টি আঘাতের চিহ্ণ। দুর্ঘটনা?,"[0.1947341793381038, 0.1620527728085868, 3.754...",positive
art4065,বিখ্যাত ব্যক্তিদের চিঠি লেখাই তাঁর শখ। উত্তর ...,state,https://ebela.in/state/letter-to-barak-obama-f...,বর্ধমানের চিঠি গেল বারাক ওবামার কাছে,"[0.08337961248920152, 0.07838917684808094, 2.0...",positive
art4076,রবিবার সকালে ঘণ্টা দু’য়েকের অনুশীলনে ম্যাচ প্...,sports,https://ebela.in/sports/mohun-bagan-prepares-f...,"কর্নার মহড়া হল, উদ্বেগ কাটছে কই","[0.07807692307692307, 0.07038461538461539, 3.3...",positive
art5106,\nএ যেন মেল গিবসনের ‘অ্যাপোক্যালিপ্টো’ সিনেম...,international,https://ebela.in/international/man-jumped-from...,২০০ ফুট ঝরনার উপর থেকে ঝাঁপ! দেখুন ভিডিও,"[0.3239477335800184, 0.24028677150786307, 3.82...",positive
art2716,সমাজতন্ত্রের তত্ত্ব খাড়া করে ‘রেড বুক’ বিলি ...,national,https://ebela.in/national/samajwadi-party-laun...,দলের গন্ধ গায়ে মাখুন! কীভাবে? জানুন...,"[0.19743031358885016, 0.14652439024390243, 3.5...",positive
art401,সিপিএমের চেয়েও কয়েক কদম এগিয়ে গেল সিপিআই। বিজে...,state,http://www.anandabazar.com/state/west-bengal-n...,কংগ্রেসের সঙ্গে ভোটের জোটেও সায় সিপিআইয়ের,"[0.2632978723404255, 0.20363089731729883, 3.58...",positive
art6686,ইরিনা শায়েক ধরা পড়ে গেলেন তাঁর ভক্তদের কাছে।...,sports,https://ebela.in/sports/irina-shayk-wearing-sa...,প্রেমিক বদলান। স্নানপোশাক বদলান না ইরিনা। দেখু...,"[0.14714285714285716, 0.14482142857142857, 2.5...",positive


## Classifier

I implement three types of classifiers:

1. A simple rule-based polarity classifier using a constructed keyword list

2. A rule-based polarity classifier using SentiWordNet scores

3. A rule-based classifier taking a weighted average of scores based on:

    a. SentiWordNet
    
    b. Independently constructed wordlists to offset bias
    
    c. More features

## Data import

This module imports the data into pandas dataframes for easier use. Basic preprocessing and cleaning of the data has been done prior, and the scripts for that can be seen [over here](https://github.com/zubairabid/CL2-Project/tree/master/datasets/news-indicnlp/newsarticles/CLEANED). 

The data that I got was not tagged for sentiment, or for anything in particular. It contained News Article headlines, content, category, and link to website version.

Due to the lack of tagged data, all the evaluation done was manual as a result.

In [6]:
def importDSets():
    '''
    Gets cleaned data
    '''

    global dframe
    
    urls = [
        'Datasets/newsarticles/CLEANED/anandabazar_articles_clean.json',
        'Datasets/newsarticles/CLEANED/ebala_articles_cleaner.json'
    ]
    
    # Resetting the dframe
    dframe = pd.DataFrame(index=['body', 'label', 'link', 'title'])
    
    for link in urls:
        c = len(dframe.columns)
        
        with open(link, 'r', encoding = 'utf-8') as f1:
            contents = f1.read()
            
            df = pd.DataFrame(json.loads(contents))
            newnums = [('art'+str(i)) for i in range(c, (c+len(df.columns)))]
            df.columns = newnums
            dframe = dframe.join(df)
            
    
# Demonstrating the result
importDSets()
dframe

,art0,art1,art2,art3,art4,art5,art6,art7,art8,art9,...,art8311,art8312,art8313,art8314,art8315,art8316,art8317,art8318,art8319,art8320
body,জঙ্গি হটাতে যুদ্ধের ডাক দিয়েছিলেন আগেই। এ বার ...,শাসকদলের গোষ্ঠীকোন্দলের জেরে সোমবার চাষিদের নথ...,শিক্ষার সর্বস্তরে শিক্ষক-শিক্ষিকাদের হাজিরায় ব...,এখন সোশ্যাল মিডিয়া ছাড়া ভাবাই যায় না। কোনও ঘট...,তদন্তের গতিপ্রকৃতি নিয়ে আদালতে প্রশ্ন ওঠার পর ...,রবিবার ফিলিপিন্সের ম্যানিলায় পোপ ফ্রান্সিসের ব...,হরকাবাহাদুর ছেত্রীর অনুগামীরা সকালেই পোস্টার স...,"বিয়ের ঘোষণার পরই শোনা গিয়েছিল, এই বিয়েতে নিমন্...",জোটের জন্য প্রস্তুত নয় কংগ্রেস। তাই জোট বাঁধার...,"যক্ষ্মার মতো উপসর্গ, আদতে নয়। তবু চিকিৎসা শুরু...",...,মাঠে খেলার পরিবর্তে গড়াগড়ি খাওয়াতেই নাকি বে...,শিক্ষকতার চাকরি পেয়েছিলেন পুরুষ পরিচয়ে। এবার ...,কটকে ডার্বিতে ফের হারের ২৪ ঘণ্টার মধ্যে পাল্ট...,কোঙ্কনি ও সিন্ধ্রি মতে বিয়ে সম্পন্ন হয়েছে দীপ...,লরি করে যাচ্ছিল পতঞ্জলির বিভিন্ন সামগ্রী। কিন...,বিরাট কোহলির ভারতীয় দলের গুরুত্বপূর্ণ সদস্য ত...,‘বাজিরাও মস্তানি’ (২০১৪) ছবিতে দিওয়ানি গানটিত...,১৭ জুন: রাশিয়া বিশ্বকাপের অন্যতম ফেভারিট মনে ...,১. রোহিত শর্মা বনাম শাজিল খান- রোহিত এগিয়ে\n২...,বলিউডে পা রাখার আগে থেকেই তাঁর প্রেমিক চরিত্র...
label,international,state,state,national,state,international,state,entertainment,national,state,...,sports,state,sports,entertainment,state,sports,entertainment,sports,sports,entertainment
link,http://www.anandabazar.com/international/donal...,http://www.anandabazar.com/state/police-camp-i...,http://www.anandabazar.com/state/choice-based-...,http://www.anandabazar.com/national/mumbai-gir...,http://www.anandabazar.com/state/two-more-rose...,http://www.anandabazar.com/international/%E0%A...,http://www.anandabazar.com/state/morcha-takes-...,http://www.anandabazar.com/entertainment/could...,http://www.anandabazar.com/national/no-allianc...,http://www.anandabazar.com/state/campaign-over...,...,https://ebela.in/sports/fifa-wc-2018-andres-gu...,https://ebela.in/state/transgender-atri-kar-wi...,https://ebela.in/sports/east-bengal-footballer...,https://ebela.in/entertainment/finally-a-dance...,https://ebela.in/state/police-seize-drugs-in-i...,https://ebela.in/sports/jasprit-bumrah-has-sti...,https://ebela.in/entertainment/what-happens-to...,https://ebela.in/sports/fifa-wc-2018-brazil-he...,https://ebela.in/sports/which-team-s-batting-i...,https://ebela.in/entertainment/salman-khan-rem...
title,শান্তি চেয়ে ট্রাম্প বেথলেহেমে,ফের ক্যাম্প চালু বেড়াবেড়িতে,পছন্দসই মিশ্র পাঠে হাজিরায় ফাঁকি নয়: পার্থ,হোয়াট‌্সঅ্যাপ গ্রুপ ছেড়ে দারুণ বার্তা দিলেন ...,রোজভ্যালির আরও দুই কর্তা গ্রেফতার,"শিশুদের বাঁচায় না কেন ঈশ্বর, প্রশ্ন খুদের",কালিম্পঙের জমি ধরে রাখতে হরকার দাবিকে সমর্থন ম...,"দীপিকা-রণবীরের বিয়ে, কেঁদে ফেললেন কে?","কংগ্রেসের সঙ্গে জোট নয়, জানিয়ে দিলেন মায়াবতী","প্রচার কম, সঠিক হিসেব নেই যক্ষ্মার",...,"‘‘নেমার ঘরে গিয়ে গড়াক’’, প্রাক্তন প্রতিপক্ষ ধ...",রূপান্তরকামী পরিচয়েই ইউপিএসসি পরীক্ষা দেবেন ত্...,"বিমানে বাগান, ট্রেনে অর্ণবরা","বিয়ের আসরে ডুয়েট ডান্স, দীপবীরের ভিডিও এল সামনে","পতঞ্জলির সামগ্রী সামনে রেখে মারাত্মক ব্যবসা, র...",কোহলির দলের এই সদস্য এখনও এক রানও করতে পারেননি...,"শ্যুটিং শেষ, সিনেমাও হিট, তারপর! বলি তারকাদের ...","নিষ্প্রভ নেমার, চল্লিশ বছর পর বিশ্বকাপে প্রথম ...","ব্যাটসম্যানদের বিচারে কে এগিয়ে, কে পিছিয়ে",হাঁচি দিয়েই প্রেমে ব্যাঘাত! সলমন শোনালেন ব্যক্...


## The Final Product




### 3. A rule-based classifier taking a weighted average of scores based on a number of features

The idea was to construct a polarity classifier with reasonable accuracy, using a makeshift "Bengali SentiWordNet" to get positive/negativity scores that could be used for classification. I had already tried a basic system incorporating an average score system, and while it was functional better could be done.

#### Classification Features
First, getting a Bengali SentiWordNet. I used a **word-level translator** [constructed for the task](bn_en_dict.ipynb) and then ran English SentiWordNet for each translation.

The first level of scoring was at the sentence level. **Sentence length** was an important feature taken into consideration. For each word, we added the SWN score to the sentence_total.

Scoring an article was done with an assumption: News Articles (almost) always follow a **pyramid structure** when distributing features. So the scores of each article were added in *weighted average*, in the form of *concave parabola*. This is not the only averaging method possible, and others including linear or cubic may be discussed.

The **title of the article** was seen to contribute a lot to the overall sentiment, so after much experimentation it was settled that the title alone should be weighted at about 12% of the full article.

At this point there were inaccurate results on 5 out of 9 articles in observation. After a lot of trial and error it was realised that the **SentiWordNet itself had problems**, primarily relating to neutral output for what one would typically assume sentimentally charged words, like: *coalition, split, arrest(ed)*. So I added a positive and negative bias, using external word lists to identify sentimentally charged words not taken into consideration by SWN. The values of the biases were arrived at by trial and error.

In [7]:
def sentiSentLev():
    
    # New frame containing much more data than before
    newframe = pd.DataFrame(index=['body', 'label', 'link', 'title', 'score', 'sentiment'])
    newframe = newframe.join(dframe)
    
    # Bias measure introduced to offset swn inaccuracy
    pos_bias = 0.125
    neg_bias = 0.15
    
    # Wordlist to store separate negative and positive
    bias_neg = []
    bias_pos = []
    
    with open('Resources/WordLists/eng_neglist', 'r', encoding = 'utf-8') as f1:
        content = f1.read()
        bias_neg = content.split('\n')
    
    with open('Resources/WordLists/eng_poslist', 'r', encoding = 'utf-8') as f1:
        content = f1.read()
        bias_pos = content.split('\n')
    
    # Analysing all articles
    for key in dframe:
        
        body = dframe[key]['title'] + '।' + dframe[key]['body']
        newframe[key]['score'] = [0, 0, 0]
        
        # Looking at the article on a sentence-by-sentence basis
        sentences = body.split('।')
        
        # The amount of weightage (in the weighted average) given to the title
        title_percentage = 12
        title_weight = max(1, int(title_percentage / 100 * len(sentences)))
        
        # Number of "sentences" being considered
        n = len(sentences) + title_weight
        # Base for the weighted average
        base = (1/6) * n * (n+1) * (2*n + 1) 

        
        i = 0
        docscore = [0, 0, 0]
        for sentence in sentences:

            # senscore keeps track of positive, negative, and objective
            senscore = [0, 0, 0]

            j = 0
            neg = 0

            # Each bengali sentence --> English sentence, word-by-word
            tempsentence = ''
            for word in tokenize(sentence):
                if word in tdict:
                    trs = tdict[word]
                    
                    tempsentence += trs + ' '
                    
            # Now, working with that "English sentence"
            for trs in tokenize(tempsentence):
                if (trs in eng_stopset) and (trs not in eng_negs):
                    continue

                # Used for negations, like "not", used to invert sentiment of a sentence
                if trs in eng_negs:
                    neg += 1

                # Snippet to pick senti_synset val
                temp = list(swn.senti_synsets(trs))
                if len(temp) > 0:
                    synword = temp[0]
                else:
                    continue

                # Adding word sentiment values to sentence score. Accomodate for bias.
                if (synword.neg_score() == 0) and (synword.pos_score() <= 0.05) and trs in bias_pos:
                    senscore[0] += min(synword.pos_score()+pos_bias, 1.0)
                else:
                    senscore[0] += synword.pos_score()
                
                if (synword.pos_score() == 0) and (synword.neg_score() <= 0.05) and trs in bias_neg:
                    senscore[1] += min(synword.neg_score()+neg_bias, 1.0)
                else:
                    senscore[1] += synword.neg_score()
                
                senscore[2] += synword.obj_score()
                # The max() is to make sure it does not go above 1


                j += 1                    

            # Normalising the score
#             senscore[0] /= j+1
#             senscore[1] /= j+1
#             senscore[2] /= j+1

            # Negation application. When there's unbalanced negators
            if neg % 2 != 0:
                senscore[0], senscore[1] = senscore[1], senscore[0]
            
            # Adding sentence scores to the document score in decreasing weighted average
            if i == 0:
                for k in range(title_weight):
                    docscore[0] += senscore[0] * ((n - k)**2)/base
                    docscore[1] += senscore[1] * ((n - k)**2)/base
                    docscore[2] += senscore[2] * ((n - k)**2)/base
            else:
                # docscore  = weighted average of senscore
                docscore[0] += senscore[0] * ((n - i)**2)/base
                docscore[1] += senscore[1] * ((n - i)**2)/base
                docscore[2] += senscore[2] * ((n - i)**2)/base
        
            # This helps with the weighted av. calculation
            if i == 0:
                i += title_weight
            else:
                i += 1
            
        print('Scores of ' + key + ':', end = ' ')
        print(docscore)

        
        if docscore[0] > docscore[1]:
            newframe[key]['sentiment'] = 'positive'
        elif docscore[0] < docscore[1]:
            newframe[key]['sentiment'] = 'negative'
        else:
            newframe[key]['sentiment'] = 'neutral'
            
    


        newframe[key]['score'] = docscore

    
    return newframe
        

weightframe = sentiSentLev()

Scores of art0: [0.28174144310823307, 0.42865402405180386, 6.996473172987975]
Scores of art1: [0.11884615384615385, 0.14173076923076927, 7.947499999999999]
Scores of art2: [0.37863291373565344, 0.12578366037270147, 7.277566950512154]
Scores of art3: [0.5192345086855069, 0.31864143116411725, 6.988802826030595]
Scores of art4: [0.20377551020408163, 0.10089285714285716, 6.59109693877551]
Scores of art5: [0.2683080070134424, 0.274427235534775, 5.222516072472238]
Scores of art6: [0.2141881210478772, 0.22887308039747062, 4.179206187895212]
Scores of art7: [0.16358543417366947, 0.1107843137254902, 4.243837535014006]
Scores of art8: [0.3405253456221198, 0.1551139784946237, 3.1448986175115206]
Scores of art9: [0.17045454545454547, 0.4035865297493205, 3.781750226517668]
Scores of art10: [0.17263753651411873, 0.1803566212268744, 3.2391918208373904]
Scores of art11: [0.21689386562804275, 0.11328140214216163, 3.6762475657254137]
Scores of art12: [0.2457107843137255, 0.03002450980392157, 2.459558823

Scores of art105: [0.24460530759522917, 0.125440285109938, 2.3490490410423224]
Scores of art106: [0.5414835800185015, 0.31418825161887143, 8.877890841813134]
Scores of art107: [0.1607736013986014, 0.3472333916083916, 4.737478146853147]
Scores of art108: [0.14369458128078821, 0.2677586206896553, 4.3222906403940895]
Scores of art109: [0.17641275136687984, 0.2288768418126216, 4.42828051153739]
Scores of art110: [0.15145425700687465, 0.12714965626652563, 3.7280407191961924]
Scores of art111: [0.19317074434998957, 0.09855173128758032, 3.498444951275139]
Scores of art112: [0.24920948616600794, 0.1661396574440053, 4.197628458498023]
Scores of art113: [0.30347699629825486, 0.18799312533051296, 5.538577472236913]
Scores of art114: [0.1108058608058608, 0.14392551892551894, 2.015262515262515]
Scores of art115: [0.1826086956521739, 0.46512422360248457, 5.057791377420533]
Scores of art116: [0.2870526348266075, 0.16126126126126122, 5.352423485128964]
Scores of art117: [0.2358805668016194, 0.02693319

Scores of art211: [0.1399859943977591, 0.04376750700280113, 4.489075630252101]
Scores of art212: [0.22475369458128083, 0.19512315270935968, 5.272536945812808]
Scores of art213: [0.34183770622559934, 0.16306553041434027, 6.756921413878511]
Scores of art214: [0.1669230769230769, 0.19942307692307693, 5.486153846153847]
Scores of art215: [0.08069828722002634, 0.05671936758893281, 4.768115942028986]
Scores of art216: [0.1734599258184164, 0.1108490566037736, 5.495565231414288]
Scores of art217: [0.05770120259019427, 0.4094299259944495, 4.426919518963923]
Scores of art218: [0.11601402688486265, 0.2438164815897136, 2.9655610753945076]
Scores of art219: [0.14388808256732788, 0.08446218351878731, 2.7794912110949848]
Scores of art220: [0.08833271991166729, 0.08329039381670963, 2.017213050107787]
Scores of art221: [0.23410931174089067, 0.401831983805668, 5.1736336032388675]
Scores of art222: [0.21563765182186234, 0.11482793522267207, 4.570344129554656]
Scores of art223: [0.413529411764706, 0.16638

Scores of art316: [0.1295927601809955, 0.18483710407239826, 3.613914027149322]
Scores of art317: [0.1348542591046875, 0.26236271581074555, 6.174444273601899]
Scores of art318: [0.1912870027752082, 0.1691200277520814, 5.229561748381128]
Scores of art319: [0.16509268292682924, 0.2445588850174216, 4.458188153310104]
Scores of art320: [0.2657071567672833, 0.15652385589094453, 6.906724683544302]
Scores of art321: [0.09349921328671329, 0.2589046328671329, 3.6233391608391616]
Scores of art322: [0.06211843711843712, 0.1536935286935287, 1.9674908424908422]
Scores of art323: [0.0964531158998253, 0.1984222481071637, 4.73165987186954]
Scores of art324: [0.17554168682470575, 0.4442131914207386, 5.598209966134494]
Scores of art325: [0.17710059468600126, 0.16574557025382322, 3.553228002556331]
Scores of art326: [0.22633003713113156, 0.11175086965018569, 4.899076607387142]
Scores of art327: [0.05727270002547617, 0.05588462287760944, 3.0562742585682394]
Scores of art328: [0.15252100840336136, 0.2543417

Scores of art423: [0.08752141633352369, 0.11236381408425955, 3.886501669375741]
Scores of art424: [0.14616724738675962, 0.19040940766550524, 3.553179442508711]
Scores of art425: [0.11316796248303097, 0.04269252128841171, 2.8590336912254717]
Scores of art426: [0.2287084398976982, 0.13911216660577275, 4.403662769455609]
Scores of art427: [0.17718591879550782, 0.17938417869924717, 4.918116746883872]
Scores of art428: [0.5020146486906923, 0.49271895255379833, 6.061560150375939]
Scores of art429: [0.12510515799989483, 0.04372732530627267, 3.421650717703349]
Scores of art430: [0.12720378532538246, 0.18157570650764845, 5.652385273528651]
Scores of art431: [0.07588242453748782, 0.04223588120740019, 3.849987828627069]
Scores of art432: [0.24584859584859586, 0.2464896214896215, 7.137210012210012]
Scores of art433: [0.05446428571428572, 0.016071428571428573, 4.008928571428572]
Scores of art434: [0.31630102040816327, 0.3279285714285715, 6.511020408163264]
Scores of art435: [0.10576923076923078, 0.

...

Scores of art7851: [0.46071428571428574, 0.3027310924369748, 4.802450980392159]
Scores of art7852: [0.21952913279132796, 0.36078139114724483, 4.336901535682024]
Scores of art7853: [0.18364394604517517, 0.317655040306489, 2.5986211988187407]
Scores of art7854: [0.9487315010570824, 0.22814859559045605, 6.008305647840531]
Scores of art7855: [0.14269657947686115, 0.15744426559356137, 3.523977196512407]
Scores of art7856: [0.5241596638655464, 0.32246498599439777, 4.949649859943978]
Scores of art7857: [0.01559233449477352, 0.15251742160278747, 3.6784407665505223]
Scores of art7858: [0.29939185616076147, 0.12182707562136434, 4.25735060814384]
Scores of art7859: [0.2187246963562753, 0.07692307692307693, 5.198532388663967]
Scores of art7860: [0.30211817168338906, 0.2538071348940914, 3.3015448319796157]
Scores of art7861: [0.14346764346764346, 0.121001221001221, 4.48870573870574]
Scores of art7862: [0.32302746931618936, 0.24881648158971365, 5.118571011104618]
Scores of art7863: [0.06333655861957

Scores of art7957: [0.14964574898785427, 0.05385627530364373, 5.8393218623481795]
Scores of art7958: [0.10573122529644269, 0.1532114624505929, 5.0639822134387344]
Scores of art7959: [0.0931447963800905, 0.22950226244343896, 2.5083031674208143]
Scores of art7960: [0.10517241379310346, 0.1740147783251232, 4.242980295566503]
Scores of art7961: [0.471107726212082, 0.18629764065335758, 3.1187937516204305]
Scores of art7962: [0.08471346476175275, 0.1291982600367148, 4.623762870141271]
Scores of art7963: [0.24442571947109154, 0.22547964739434798, 4.66897848068447]
Scores of art7964: [0.060902503293807635, 0.1234189723320158, 2.5787878787878786]
Scores of art7965: [0.22698889916743759, 0.20579324699352453, 4.994218316373728]
Scores of art7966: [0.19495320197044338, 0.3654240342234898, 3.6402806585429084]
Scores of art7967: [0.14514652014652013, 0.052655677655677656, 3.8107448107448105]
Scores of art7968: [0.15054023544589584, 0.1485203999354943, 2.5619658119658117]
Scores of art7969: [0.585504

Scores of art8061: [0.24494773519163765, 0.1180226480836237, 5.157273519163765]
Scores of art8062: [0.37942307692307703, 0.12865384615384615, 3.8473076923076928]
Scores of art8063: [0.5452935222672065, 0.5128542510121457, 10.00344129554656]
Scores of art8064: [0.08364352783366867, 0.1338615023474178, 2.629040912139504]
Scores of art8065: [0.14228521332554064, 0.2047486849795441, 2.853462887200467]
Scores of art8066: [0.04989878542510121, 0.1661842105263158, 2.4303643724696355]
Scores of art8067: [0.08936100131752304, 0.0630171277997365, 3.268280632411068]
Scores of art8068: [0.15879612714651076, 0.15268724881256854, 4.009755206430399]
Scores of art8069: [0.20428681410974245, 0.2127204647256439, 2.8139557670772692]
Scores of art8070: [0.0748353096179183, 0.0714756258234519, 3.709189723320158]
Scores of art8071: [0.16731350045303534, 0.2577016007248566, 4.77808819087889]
Scores of art8072: [0.34566532258064514, 0.025403225806451615, 4.109274193548388]
Scores of art8073: [0.07515527950310

Scores of art8165: [0.07961672473867595, 0.1587979094076655, 2.4828397212543556]
Scores of art8166: [0.2530225431342093, 0.3624480500398808, 2.744337979094077]
Scores of art8167: [0.10557491289198608, 0.20387630662020909, 3.340461672473868]
Scores of art8168: [0.12865696029655646, 0.042140278997171016, 2.7950626768120186]
Scores of art8169: [0.23767952176131812, 0.1666736531311511, 2.3276864474739374]
Scores of art8170: [0.19490418118466898, 0.19459930313588852, 2.5286149825783975]
Scores of art8171: [0.14903455284552847, 0.14364385727190607, 3.601405826558266]
Scores of art8172: [0.2275696864111498, 0.19343205574912892, 4.4887195121951216]
Scores of art8173: [0.2664504932407745, 0.17567409572524662, 3.598456339057363]
Scores of art8174: [0.11275910364145661, 0.44514005602240897, 4.655602240896359]
Scores of art8175: [0.16369199853854585, 0.21719766167336502, 3.771940080379979]
Scores of art8176: [0.21072398190045252, 0.21197285067873312, 4.624570135746608]
Scores of art8177: [0.183907

Scores of art8269: [0.1407563025210084, 0.1023809523809524, 4.094887955182073]
Scores of art8270: [0.2736734693877551, 0.14389795918367346, 4.173877551020409]
Scores of art8271: [0.20253357753357754, 0.32341269841269843, 3.8214285714285716]
Scores of art8272: [0.2718466424682396, 0.30118939590355204, 4.914765361680064]
Scores of art8273: [0.262878395189716, 0.07916182873730043, 3.626083350611652]
Scores of art8274: [0.22138510223953262, 0.14773369036027262, 3.6457582765335923]
Scores of art8275: [0.1828778280542987, 0.043561085972850695, 3.1922398190045254]
Scores of art8276: [0.09359072375127422, 0.09046567971175845, 3.401301866805715]
Scores of art8277: [0.07980295566502463, 0.044334975369458136, 1.9748768472906404]
Scores of art8278: [0.21403940886699513, 0.00788177339901478, 2.941625615763548]
Scores of art8279: [0.528359866716031, 0.26908243860298653, 4.064227755152413]
Scores of art8280: [0.40905484041615237, 0.15118389858735481, 3.2697422559219422]
Scores of art8281: [0.25320197

In [8]:
weightframe

,art0,art1,art2,art3,art4,art5,art6,art7,art8,art9,...,art8311,art8312,art8313,art8314,art8315,art8316,art8317,art8318,art8319,art8320
body,জঙ্গি হটাতে যুদ্ধের ডাক দিয়েছিলেন আগেই। এ বার ...,শাসকদলের গোষ্ঠীকোন্দলের জেরে সোমবার চাষিদের নথ...,শিক্ষার সর্বস্তরে শিক্ষক-শিক্ষিকাদের হাজিরায় ব...,এখন সোশ্যাল মিডিয়া ছাড়া ভাবাই যায় না। কোনও ঘট...,তদন্তের গতিপ্রকৃতি নিয়ে আদালতে প্রশ্ন ওঠার পর ...,রবিবার ফিলিপিন্সের ম্যানিলায় পোপ ফ্রান্সিসের ব...,হরকাবাহাদুর ছেত্রীর অনুগামীরা সকালেই পোস্টার স...,"বিয়ের ঘোষণার পরই শোনা গিয়েছিল, এই বিয়েতে নিমন্...",জোটের জন্য প্রস্তুত নয় কংগ্রেস। তাই জোট বাঁধার...,"যক্ষ্মার মতো উপসর্গ, আদতে নয়। তবু চিকিৎসা শুরু...",...,মাঠে খেলার পরিবর্তে গড়াগড়ি খাওয়াতেই নাকি বে...,শিক্ষকতার চাকরি পেয়েছিলেন পুরুষ পরিচয়ে। এবার ...,কটকে ডার্বিতে ফের হারের ২৪ ঘণ্টার মধ্যে পাল্ট...,কোঙ্কনি ও সিন্ধ্রি মতে বিয়ে সম্পন্ন হয়েছে দীপ...,লরি করে যাচ্ছিল পতঞ্জলির বিভিন্ন সামগ্রী। কিন...,বিরাট কোহলির ভারতীয় দলের গুরুত্বপূর্ণ সদস্য ত...,‘বাজিরাও মস্তানি’ (২০১৪) ছবিতে দিওয়ানি গানটিত...,১৭ জুন: রাশিয়া বিশ্বকাপের অন্যতম ফেভারিট মনে ...,১. রোহিত শর্মা বনাম শাজিল খান- রোহিত এগিয়ে\n২...,বলিউডে পা রাখার আগে থেকেই তাঁর প্রেমিক চরিত্র...
label,international,state,state,national,state,international,state,entertainment,national,state,...,sports,state,sports,entertainment,state,sports,entertainment,sports,sports,entertainment
link,http://www.anandabazar.com/international/donal...,http://www.anandabazar.com/state/police-camp-i...,http://www.anandabazar.com/state/choice-based-...,http://www.anandabazar.com/national/mumbai-gir...,http://www.anandabazar.com/state/two-more-rose...,http://www.anandabazar.com/international/%E0%A...,http://www.anandabazar.com/state/morcha-takes-...,http://www.anandabazar.com/entertainment/could...,http://www.anandabazar.com/national/no-allianc...,http://www.anandabazar.com/state/campaign-over...,...,https://ebela.in/sports/fifa-wc-2018-andres-gu...,https://ebela.in/state/transgender-atri-kar-wi...,https://ebela.in/sports/east-bengal-footballer...,https://ebela.in/entertainment/finally-a-dance...,https://ebela.in/state/police-seize-drugs-in-i...,https://ebela.in/sports/jasprit-bumrah-has-sti...,https://ebela.in/entertainment/what-happens-to...,https://ebela.in/sports/fifa-wc-2018-brazil-he...,https://ebela.in/sports/which-team-s-batting-i...,https://ebela.in/entertainment/salman-khan-rem...
title,শান্তি চেয়ে ট্রাম্প বেথলেহেমে,ফের ক্যাম্প চালু বেড়াবেড়িতে,পছন্দসই মিশ্র পাঠে হাজিরায় ফাঁকি নয়: পার্থ,হোয়াট‌্সঅ্যাপ গ্রুপ ছেড়ে দারুণ বার্তা দিলেন ...,রোজভ্যালির আরও দুই কর্তা গ্রেফতার,"শিশুদের বাঁচায় না কেন ঈশ্বর, প্রশ্ন খুদের",কালিম্পঙের জমি ধরে রাখতে হরকার দাবিকে সমর্থন ম...,"দীপিকা-রণবীরের বিয়ে, কেঁদে ফেললেন কে?","কংগ্রেসের সঙ্গে জোট নয়, জানিয়ে দিলেন মায়াবতী","প্রচার কম, সঠিক হিসেব নেই যক্ষ্মার",...,"‘‘নেমার ঘরে গিয়ে গড়াক’’, প্রাক্তন প্রতিপক্ষ ধ...",রূপান্তরকামী পরিচয়েই ইউপিএসসি পরীক্ষা দেবেন ত্...,"বিমানে বাগান, ট্রেনে অর্ণবরা","বিয়ের আসরে ডুয়েট ডান্স, দীপবীরের ভিডিও এল সামনে","পতঞ্জলির সামগ্রী সামনে রেখে মারাত্মক ব্যবসা, র...",কোহলির দলের এই সদস্য এখনও এক রানও করতে পারেননি...,"শ্যুটিং শেষ, সিনেমাও হিট, তারপর! বলি তারকাদের ...","নিষ্প্রভ নেমার, চল্লিশ বছর পর বিশ্বকাপে প্রথম ...","ব্যাটসম্যানদের বিচারে কে এগিয়ে, কে পিছিয়ে",হাঁচি দিয়েই প্রেমে ব্যাঘাত! সলমন শোনালেন ব্যক্...
score,"[0.28174144310823307, 0.42865402405180386, 6.9...","[0.11884615384615385, 0.14173076923076927, 7.9...","[0.37863291373565344, 0.12578366037270147, 7.2...","[0.5192345086855069, 0.31864143116411725, 6.98...","[0.20377551020408163, 0.10089285714285716, 6.5...","[0.2683080070134424, 0.274427235534775, 5.2225...","[0.2141881210478772, 0.22887308039747062, 4.17...","[0.16358543417366947, 0.1107843137254902, 4.24...","[0.3405253456221198, 0.1551139784946237, 3.144...","[0.17045454545454547, 0.4035865297493205, 3.78...",...,"[0.350659111933395, 0.09958950046253469, 4.517...","[0.11036751361161527, 0.14016722841586726, 3.2...","[0.08374271768225479, 0.05385490473941112, 2.3...","[0.009768009768009768, 0.08714896214896214, 3....","[0.2242914979757085, 0.25126

## Results

6/7 out of the first 9 articles are tagged correctly. The other two are closer to correct than by any other method, and largely have sufficient reasoning for their failures.

1. **Failure**: SentiWordNet classification is wrong. Eg: arrest(ed) is neutral, and not negative (art4)

**Fix**: Add a bias, using custom positive and negative wordlists. It does not eliminate the problem, but reduces it massively from 0.7+ 0.1- to 0.2+ 0.1-

2. **Failure**: Article is *not* pyramid-shaped. Eg: art0. While it *is* largely negative, the title and the final sentiment conveyed are positive. But the article is majorly negative.

**Fix**: Not much that can be done, but increasing weightage of the title seems to help. It cannot however be increased too much.

3. **Failure**: Words not in dictionary. Eg: তুলোধনা

**Fix**: Use Google Translate to keep building dictionary vocab


## Other Approaches

### 1. A simple rule-based polarity classifier using a constructed keyword list

A polarity classifier. Based on certain rules (keyword appearances), it classifies articles as positive, negative, or neutral.

In [9]:
def simplePol():
    '''
    
    '''
    
    # Hardcoded wordlists
    poslist = ['ভালো', 'ভাল', 'খুশি', 'সৌখিন', 'উজ্জ্বল', 'দারুন', 'আনন্দের', 'আনন্দ', 'happiness']
    neglist = ['খারাপ', 'জঘন্য', 'অশৌখিন', 'দুঃখ']
    negator = ['not', 'n\'t', 'none', 'never', 'no', 'non']
    
    # New frame containing much more data than before
    newframe = pd.DataFrame(index=['body', 'label', 'link', 'title', 'score', 'sentiment'])
    newframe = newframe.join(dframe)
    
    # Iterating over columns
    for key in dframe:

        # Variables to track
        
        # Current sentiment score
        sent = 0
        
        # Previous sentiment score
        last = 0
        
        # Total score of document
        total = 0
        
        # Negator variable
        neg = False

        body = dframe[key]['body']
        for word in tokenize(body):
            
            # 
            if word in poslist:
                # Positive: add a positive sentiment score
                sent = 1
                # print('Identified {0} as positive, adding 1 to score'.format(word))

            elif word in neglist:
                # Negative: add a negative sentiment score
                sent = -1
                # print('Identified {0} as positive, subtracting 1 from score'.format(word))

            elif word in negator and last != 0:
                # A negative word: mark it as negative, when adding score add twice the last
                neg = True
                # print('Identified {0} as a negator, inverting previous score'.format(word))


            if neg:
                total += -2*last
            else:
                total += sent


            # Reset variables
            neg = False
            last = sent
            sent = 0

#         print(key + ": ", str(total))
        newframe[key]['score'] = total
        if total > 0:
            newframe[key]['sentiment'] = 'positive'
        elif total == 0:
            newframe[key]['sentiment'] = 'neutral'
        else:
            newframe[key]['sentiment'] = 'negative'
            
    
    return newframe

s1frame = simplePol()

In [10]:
s1frame

,art0,art1,art2,art3,art4,art5,art6,art7,art8,art9,...,art8311,art8312,art8313,art8314,art8315,art8316,art8317,art8318,art8319,art8320
body,জঙ্গি হটাতে যুদ্ধের ডাক দিয়েছিলেন আগেই। এ বার ...,শাসকদলের গোষ্ঠীকোন্দলের জেরে সোমবার চাষিদের নথ...,শিক্ষার সর্বস্তরে শিক্ষক-শিক্ষিকাদের হাজিরায় ব...,এখন সোশ্যাল মিডিয়া ছাড়া ভাবাই যায় না। কোনও ঘট...,তদন্তের গতিপ্রকৃতি নিয়ে আদালতে প্রশ্ন ওঠার পর ...,রবিবার ফিলিপিন্সের ম্যানিলায় পোপ ফ্রান্সিসের ব...,হরকাবাহাদুর ছেত্রীর অনুগামীরা সকালেই পোস্টার স...,"বিয়ের ঘোষণার পরই শোনা গিয়েছিল, এই বিয়েতে নিমন্...",জোটের জন্য প্রস্তুত নয় কংগ্রেস। তাই জোট বাঁধার...,"যক্ষ্মার মতো উপসর্গ, আদতে নয়। তবু চিকিৎসা শুরু...",...,মাঠে খেলার পরিবর্তে গড়াগড়ি খাওয়াতেই নাকি বে...,শিক্ষকতার চাকরি পেয়েছিলেন পুরুষ পরিচয়ে। এবার ...,কটকে ডার্বিতে ফের হারের ২৪ ঘণ্টার মধ্যে পাল্ট...,কোঙ্কনি ও সিন্ধ্রি মতে বিয়ে সম্পন্ন হয়েছে দীপ...,লরি করে যাচ্ছিল পতঞ্জলির বিভিন্ন সামগ্রী। কিন...,বিরাট কোহলির ভারতীয় দলের গুরুত্বপূর্ণ সদস্য ত...,‘বাজিরাও মস্তানি’ (২০১৪) ছবিতে দিওয়ানি গানটিত...,১৭ জুন: রাশিয়া বিশ্বকাপের অন্যতম ফেভারিট মনে ...,১. রোহিত শর্মা বনাম শাজিল খান- রোহিত এগিয়ে\n২...,বলিউডে পা রাখার আগে থেকেই তাঁর প্রেমিক চরিত্র...
label,international,state,state,national,state,international,state,entertainment,national,state,...,sports,state,sports,entertainment,state,sports,entertainment,sports,sports,entertainment
link,http://www.anandabazar.com/international/donal...,http://www.anandabazar.com/state/police-camp-i...,http://www.anandabazar.com/state/choice-based-...,http://www.anandabazar.com/national/mumbai-gir...,http://www.anandabazar.com/state/two-more-rose...,http://www.anandabazar.com/international/%E0%A...,http://www.anandabazar.com/state/morcha-takes-...,http://www.anandabazar.com/entertainment/could...,http://www.anandabazar.com/national/no-allianc...,http://www.anandabazar.com/state/campaign-over...,...,https://ebela.in/sports/fifa-wc-2018-andres-gu...,https://ebela.in/state/transgender-atri-kar-wi...,https://ebela.in/sports/east-bengal-footballer...,https://ebela.in/entertainment/finally-a-dance...,https://ebela.in/state/police-seize-drugs-in-i...,https://ebela.in/sports/jasprit-bumrah-has-sti...,https://ebela.in/entertainment/what-happens-to...,https://ebela.in/sports/fifa-wc-2018-brazil-he...,https://ebela.in/sports/which-team-s-batting-i...,https://ebela.in/entertainment/salman-khan-rem...
title,শান্তি চেয়ে ট্রাম্প বেথলেহেমে,ফের ক্যাম্প চালু বেড়াবেড়িতে,পছন্দসই মিশ্র পাঠে হাজিরায় ফাঁকি নয়: পার্থ,হোয়াট‌্সঅ্যাপ গ্রুপ ছেড়ে দারুণ বার্তা দিলেন ...,রোজভ্যালির আরও দুই কর্তা গ্রেফতার,"শিশুদের বাঁচায় না কেন ঈশ্বর, প্রশ্ন খুদের",কালিম্পঙের জমি ধরে রাখতে হরকার দাবিকে সমর্থন ম...,"দীপিকা-রণবীরের বিয়ে, কেঁদে ফেললেন কে?","কংগ্রেসের সঙ্গে জোট নয়, জানিয়ে দিলেন মায়াবতী","প্রচার কম, সঠিক হিসেব নেই যক্ষ্মার",...,"‘‘নেমার ঘরে গিয়ে গড়াক’’, প্রাক্তন প্রতিপক্ষ ধ...",রূপান্তরকামী পরিচয়েই ইউপিএসসি পরীক্ষা দেবেন ত্...,"বিমানে বাগান, ট্রেনে অর্ণবরা","বিয়ের আসরে ডুয়েট ডান্স, দীপবীরের ভিডিও এল সামনে","পতঞ্জলির সামগ্রী সামনে রেখে মারাত্মক ব্যবসা, র...",কোহলির দলের এই সদস্য এখনও এক রানও করতে পারেননি...,"শ্যুটিং শেষ, সিনেমাও হিট, তারপর! বলি তারকাদের ...","নিষ্প্রভ নেমার, চল্লিশ বছর পর বিশ্বকাপে প্রথম ...","ব্যাটসম্যানদের বিচারে কে এগিয়ে, কে পিছিয়ে",হাঁচি দিয়েই প্রেমে ব্যাঘাত! সলমন শোনালেন ব্যক্...
score,0,1,2,1,0,0,0,2,0,0,...,0,0,2,0,0,1,-1,0,0,0
sentiment,neutral,positive,positive,positive,neutral,neutral,neutral,positive,neutral,neutral,...,neutral,neutral,positive,neutral,neutral,positive,negative,neutral,neutral,neutral


### 2. A rule-based polarity classifier using SentiWordNet scores

In [11]:
def sentiAnalysis():
    '''
    SentiWordNet
    Pipeline:
        Use translator
        Create a SentiWordNet for Bengali using translation from beng to eng and swn on eng 
    '''
    
    total = 0
    cn = 0
    
    newframe = pd.DataFrame(index=['body', 'label', 'link', 'title', 'score', 'sentiment'])
    newframe = newframe.join(dframe)
    
    for key in dframe:
        
        subj = 0.0
        c = 0
        
        body = dframe[key]['body']
        newframe[key]['score'] = [0, 0, 1]
        
        positive = 0.0
        negative = 0.0
        objective = 0.0

        for word in tokenize(body):
            
            if word in tdict:
                trs = tdict[word]
                
                if (trs in eng_stopset) and (trs not in eng_negs):
                    continue
                
                temp = list(swn.senti_synsets(trs))
                if len(temp) > 0:
                    synword = temp[0]
                else:
                    continue
                
                c += 1
                subj += (1 - synword.obj_score())
                objective += synword.obj_score()
                positive += synword.pos_score()
                negative += synword.neg_score()

            else:
                pass
        
            
        if c != 0:
            total += subj/c
            cn += 1
        
            newframe[key]['score'] = [positive/c, negative/c, objective/c]
            
            if newframe[key]['score'][0] > newframe[key]['score'][1]:
                newframe[key]['sentiment'] = 'positive'
            elif newframe[key]['score'][0] < newframe[key]['score'][1]:
                newframe[key]['sentiment'] = 'negative'
            else:
                newframe[key]['sentiment'] = 'neutral'
        
            
    return newframe

    
nonframe = sentiAnalysis()

In [12]:
nonframe

,art0,art1,art2,art3,art4,art5,art6,art7,art8,art9,...,art8311,art8312,art8313,art8314,art8315,art8316,art8317,art8318,art8319,art8320
body,জঙ্গি হটাতে যুদ্ধের ডাক দিয়েছিলেন আগেই। এ বার ...,শাসকদলের গোষ্ঠীকোন্দলের জেরে সোমবার চাষিদের নথ...,শিক্ষার সর্বস্তরে শিক্ষক-শিক্ষিকাদের হাজিরায় ব...,এখন সোশ্যাল মিডিয়া ছাড়া ভাবাই যায় না। কোনও ঘট...,তদন্তের গতিপ্রকৃতি নিয়ে আদালতে প্রশ্ন ওঠার পর ...,রবিবার ফিলিপিন্সের ম্যানিলায় পোপ ফ্রান্সিসের ব...,হরকাবাহাদুর ছেত্রীর অনুগামীরা সকালেই পোস্টার স...,"বিয়ের ঘোষণার পরই শোনা গিয়েছিল, এই বিয়েতে নিমন্...",জোটের জন্য প্রস্তুত নয় কংগ্রেস। তাই জোট বাঁধার...,"যক্ষ্মার মতো উপসর্গ, আদতে নয়। তবু চিকিৎসা শুরু...",...,মাঠে খেলার পরিবর্তে গড়াগড়ি খাওয়াতেই নাকি বে...,শিক্ষকতার চাকরি পেয়েছিলেন পুরুষ পরিচয়ে। এবার ...,কটকে ডার্বিতে ফের হারের ২৪ ঘণ্টার মধ্যে পাল্ট...,কোঙ্কনি ও সিন্ধ্রি মতে বিয়ে সম্পন্ন হয়েছে দীপ...,লরি করে যাচ্ছিল পতঞ্জলির বিভিন্ন সামগ্রী। কিন...,বিরাট কোহলির ভারতীয় দলের গুরুত্বপূর্ণ সদস্য ত...,‘বাজিরাও মস্তানি’ (২০১৪) ছবিতে দিওয়ানি গানটিত...,১৭ জুন: রাশিয়া বিশ্বকাপের অন্যতম ফেভারিট মনে ...,১. রোহিত শর্মা বনাম শাজিল খান- রোহিত এগিয়ে\n২...,বলিউডে পা রাখার আগে থেকেই তাঁর প্রেমিক চরিত্র...
label,international,state,state,national,state,international,state,entertainment,national,state,...,sports,state,sports,entertainment,state,sports,entertainment,sports,sports,entertainment
link,http://www.anandabazar.com/international/donal...,http://www.anandabazar.com/state/police-camp-i...,http://www.anandabazar.com/state/choice-based-...,http://www.anandabazar.com/national/mumbai-gir...,http://www.anandabazar.com/state/two-more-rose...,http://www.anandabazar.com/international/%E0%A...,http://www.anandabazar.com/state/morcha-takes-...,http://www.anandabazar.com/entertainment/could...,http://www.anandabazar.com/national/no-allianc...,http://www.anandabazar.com/state/campaign-over...,...,https://ebela.in/sports/fifa-wc-2018-andres-gu...,https://ebela.in/state/transgender-atri-kar-wi...,https://ebela.in/sports/east-bengal-footballer...,https://ebela.in/entertainment/finally-a-dance...,https://ebela.in/state/police-seize-drugs-in-i...,https://ebela.in/sports/jasprit-bumrah-has-sti...,https://ebela.in/entertainment/what-happens-to...,https://ebela.in/sports/fifa-wc-2018-brazil-he...,https://ebela.in/sports/which-team-s-batting-i...,https://ebela.in/entertainment/salman-khan-rem...
title,শান্তি চেয়ে ট্রাম্প বেথলেহেমে,ফের ক্যাম্প চালু বেড়াবেড়িতে,পছন্দসই মিশ্র পাঠে হাজিরায় ফাঁকি নয়: পার্থ,হোয়াট‌্সঅ্যাপ গ্রুপ ছেড়ে দারুণ বার্তা দিলেন ...,রোজভ্যালির আরও দুই কর্তা গ্রেফতার,"শিশুদের বাঁচায় না কেন ঈশ্বর, প্রশ্ন খুদের",কালিম্পঙের জমি ধরে রাখতে হরকার দাবিকে সমর্থন ম...,"দীপিকা-রণবীরের বিয়ে, কেঁদে ফেললেন কে?","কংগ্রেসের সঙ্গে জোট নয়, জানিয়ে দিলেন মায়াবতী","প্রচার কম, সঠিক হিসেব নেই যক্ষ্মার",...,"‘‘নেমার ঘরে গিয়ে গড়াক’’, প্রাক্তন প্রতিপক্ষ ধ...",রূপান্তরকামী পরিচয়েই ইউপিএসসি পরীক্ষা দেবেন ত্...,"বিমানে বাগান, ট্রেনে অর্ণবরা","বিয়ের আসরে ডুয়েট ডান্স, দীপবীরের ভিডিও এল সামনে","পতঞ্জলির সামগ্রী সামনে রেখে মারাত্মক ব্যবসা, র...",কোহলির দলের এই সদস্য এখনও এক রানও করতে পারেননি...,"শ্যুটিং শেষ, সিনেমাও হিট, তারপর! বলি তারকাদের ...","নিষ্প্রভ নেমার, চল্লিশ বছর পর বিশ্বকাপে প্রথম ...","ব্যাটসম্যানদের বিচারে কে এগিয়ে, কে পিছিয়ে",হাঁচি দিয়েই প্রেমে ব্যাঘাত! সলমন শোনালেন ব্যক্...
score,"[0.030109489051094892, 0.04927007299270073, 0....","[0.02403846153846154, 0.009615384615384616, 0....","[0.034254807692307696, 0.030048076923076924, 0...","[0.050335570469798654, 0.06375838926174497, 0....","[0.027439024390243903, 0.026422764227642278, 0...","[0.020522388059701493, 0.048507462686567165, 0...","[0.04632352941176471, 0.06911764705882353, 0.8...","[0.05053191489361702, 0.031914893617021274, 0....","[0.03537735849056604, 0.08077830188679246, 0.8...","[0.024456521739130436, 0.06929347826086957, 0....",...,"[0.04066265060240964, 0.0572289156626506, 0.90...","[0.0351123595505618, 0.060393258426966294, 0.9...","[0.031725888324873094, 0.038705583756345176, 0...","[0.011363636363636364, 0.030303030303030304, 0...","[0.027960526315789474, 0.054